In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from typing import List

import time

from statistics import mean

from xgboost import XGBClassifier

from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import silhouette_score


from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_pickle('../data/processed/train.pkl')
df

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.522699,-2.3978,7.8784,8.5635,12.780300,-1.0914
train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.430500,2.0339,8.1267,8.7889,18.356001,1.9518
train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.604200,3.1417,-6.5213,8.2675,14.722200,0.3965
train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.034700,-1.2706,-2.9275,10.2922,17.969700,-8.9996
train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.287600,-1.5121,3.9267,9.5031,17.997400,-8.8104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
train_199995,0,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,-0.6209,...,6.1415,13.2305,3.9901,0.9388,18.024900,-1.7939,2.1661,8.5326,16.666000,-17.8661
train_199996,0,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,-3.6797,...,4.9611,4.6549,0.6998,1.8341,22.271700,1.7337,-2.1651,6.7419,15.905400,0.3388
train_199997,0,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,0.1202,...,4.0651,5.4414,3.1032,4.8793,23.531099,-1.5736,1.2832,8.7155,13.832900,4.1995


In [4]:
X=df.drop("target", axis=1)
y=df["target"]

In [5]:
skf = StratifiedKFold(n_splits=7,shuffle=False)
skf.get_n_splits(X, y)

7

In [6]:
#Genero lista de resultados
results=[]
lista_name=[]
tiempo_ejecucion=[]

In [17]:
lgr = LogisticRegression()

In [18]:
#==================================LogisticRegression================================================

contador=0
recall_prom=[] #Incializo recall_prom lo hice para copiar y pegar
inicio = time.time() #Inicio timer para costo

for train_index, test_index in skf.split(X, y):
    
    X_train=X.iloc[train_index] #obtengo xtrain
    X_test=X.iloc[test_index] #obtengo xtest
    y_train=y.iloc[train_index] #obtengo ytrain
    y_test=y.iloc[test_index] #obtengo ytest
    
    lgr.fit(X_train,y_train) #Entreno modelo
    
    y_pred=np.asarray(lgr.predict(X_test)) #Calculo la spreducciones
    
    cm=confusion_matrix(y_test, y_pred) #Calculo matriz de confusion
    precision = precision_score(y_test, y_pred,average='binary') #Calculo precision
    recall = recall_score(y_test, y_pred,average='binary') #Calculo recall
    
    if contador==5:
        
        print("El recall del fold 5 es",recall )
        print(cm)
        print("El precision del fold 5 es",precision)
    contador+=1
    
    lista=[precision,recall]
    recall_prom.append(recall)

results.append(mean(recall_prom)) #Guardo recall

print()
print("El valor promedio del recall es", mean(recall_prom)) #Calculo promedio

lista_name.append("Fulldata_LogisticRegression") #Agrego nombre a lista

fin = time.time() #Finalizo tiempo para calculo de costo
tiempo_ejecucion.append(fin-inicio) #Guardo tiempo de ejecución

print("el tiempo fue de ", fin-inicio)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

El recall del fold 5 es 0.22326715430163707
[[25408   292]
 [ 2230   641]]
El precision del fold 5 es 0.6870310825294748

El valor promedio del recall es 0.23042063887083578
el tiempo fue de  36.35167574882507


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
lgr2 = LogisticRegression(max_iter= 23980, penalty="none", solver= 'lbfgs')

In [16]:
#==================================LogisticRegression================================================

contador=0
recall_prom=[] #Incializo recall_prom lo hice para copiar y pegar
inicio = time.time() #Inicio timer para costo

for train_index, test_index in skf.split(X, y):
    
    X_train=X.iloc[train_index] #obtengo xtrain
    X_test=X.iloc[test_index] #obtengo xtest
    y_train=y.iloc[train_index] #obtengo ytrain
    y_test=y.iloc[test_index] #obtengo ytest
    
    lgr2.fit(X_train,y_train) #Entreno modelo
    
    y_pred=np.asarray(lgr2.predict(X_test)) #Calculo la spreducciones
    
    cm=confusion_matrix(y_test, y_pred) #Calculo matriz de confusion
    precision = precision_score(y_test, y_pred,average='binary') #Calculo precision
    recall = recall_score(y_test, y_pred,average='binary') #Calculo recall
    
    if contador==5:
        
        print("El recall del fold 5 es",recall )
        print(cm)
        print("El precision del fold 5 es",precision)
    contador+=1
    
    lista=[precision,recall]
    recall_prom.append(recall)

results.append(mean(recall_prom)) #Guardo recall

print()
print("El valor promedio del recall es", mean(recall_prom)) #Calculo promedio

lista_name.append("Fulldata_LogisticRegression2") #Agrego nombre a lista

fin = time.time() #Finalizo tiempo para calculo de costo
tiempo_ejecucion.append(fin-inicio) #Guardo tiempo de ejecución

print("el tiempo fue de ", fin-inicio)

El recall del fold 5 es 0.26436781609195403
[[25366   334]
 [ 2112   759]]
El precision del fold 5 es 0.6944190301921317

El valor promedio del recall es 0.26848402933793386
el tiempo fue de  742.0456206798553
